####first let's import the libraries 

In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, HTML
from openai import OpenAI
import google.generativeai as genai
# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to GEMINI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to GEMINI
.  


In [4]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('GEMINI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook


In [6]:
# Configure the Gemini API with the API key
genai.configure(api_key=api_key)

# Let's make a quick call to a Frontier model to get started, as a preview!

In [7]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.
# Example: Generate a simple response from Gemini
model = genai.GenerativeModel('models/gemini-2.5-flash-lite')
prompt = "Hello, Gemini! This is my first ever message to you! Hi!"
response = model.generate_content(prompt)
print(response.text)

Hello there! It's wonderful to hear from you! Welcome! I'm excited to be your first ever message recipient.

How can I help you today? What's on your mind? I'm ready for anything you'd like to talk about, ask about, or create!


## OK onwards with our first project

In [ ]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        self.favicon = None
        self.images = []

        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract title
            self.title = soup.title.string if soup.title else "No title found"

            # Extract favicon
            favicon_link = soup.find('link', rel='icon') or soup.find('link', rel='shortcut icon')
            if favicon_link and favicon_link.get('href'):
                favicon_url = favicon_link['href']
                if favicon_url.startswith('//'):
                    favicon_url = 'https:' + favicon_url
                elif favicon_url.startswith('/'):
                    from urllib.parse import urlparse
                    parsed = urlparse(url)
                    favicon_url = f"{parsed.scheme}://{parsed.netloc}{favicon_url}"
                self.favicon = favicon_url

            # Extract main images (limit to first few)
            img_tags = soup.find_all('img', limit=5)
            for img in img_tags:
                img_src = img.get('src', '')
                if img_src:
                    if img_src.startswith('//'):
                        img_src = 'https:' + img_src
                    elif img_src.startswith('/'):
                        from urllib.parse import urlparse
                        parsed = urlparse(url)
                        img_src = f"{parsed.scheme}://{parsed.netloc}{img_src}"
                    elif not img_src.startswith('http'):
                        from urllib.parse import urlparse, urljoin
                        parsed = urlparse(url)
                        img_src = urljoin(url, img_src)

                    # Only add reasonable sized images
                    if img.get('width', '').isdigit() and int(img.get('width', 0)) > 100:
                        self.images.append({
                            'url': img_src,
                            'alt': img.get('alt', ''),
                            'width': img.get('width', ''),
                            'height': img.get('height', '')
                        })

            # Remove irrelevant elements for text extraction
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True) if soup.body else ""

        except Exception as e:
            self.title = "Error loading website"
            self.text = f"Could not load content from {url}. Error: {str(e)}"

In [9]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://www.deeplearning.ai/the-batch/new-technique-auto-selects-training-examples-to-speed-up-fine-tuning/")
print(ed.title)
print(ed.text)

New Technique Auto-Selects Training Examples to Speed Up Fine-Tuning
✨ New course! Enroll in
Building and Evaluating Data Agents
Explore Courses
AI Newsletter
The Batch
Andrew's Letter
Data Points
ML Research
Blog
✨ AI Dev x NYC
Community
Forum
Events
Ambassadors
Ambassador Spotlight
Resources
Company
About
Careers
Contact
Start Learning
Weekly Issues
Andrew's Letters
Data Points
ML Research
Business
Science
Culture
Hardware
AI Careers
About
Subscribe
The Batch
Machine Learning Research
Article
Faster Reinforcement Learning
New technique auto-selects training examples to speed up fine-tuning
Machine Learning Research
Large Language Models (LLMs)
Reinforcement Learning
Published
Sep 24, 2025
Reading time
2
min read
Share
Loading the
Elevenlabs Text to Speech
AudioNative Player...
Fine-tuning large language models via reinforcement learning is computationally expensive, but researchers found a way to streamline the process.
What’s new:
Qinsi Wang and colleagues at UC Berkeley and Duke Un

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like gemini have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [10]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [11]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [12]:
print(user_prompt_for(ed))

You are looking at a website titled New Technique Auto-Selects Training Examples to Speed Up Fine-Tuning
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

✨ New course! Enroll in
Building and Evaluating Data Agents
Explore Courses
AI Newsletter
The Batch
Andrew's Letter
Data Points
ML Research
Blog
✨ AI Dev x NYC
Community
Forum
Events
Ambassadors
Ambassador Spotlight
Resources
Company
About
Careers
Contact
Start Learning
Weekly Issues
Andrew's Letters
Data Points
ML Research
Business
Science
Culture
Hardware
AI Careers
About
Subscribe
The Batch
Machine Learning Research
Article
Faster Reinforcement Learning
New technique auto-selects training examples to speed up fine-tuning
Machine Learning Research
Large Language Models (LLMs)
Reinforcement Learning
Published
Sep 24, 2025
Reading time
2
min read
Share
Loading the
Elevenlabs Text to Speech
AudioNative Player...
Fine-

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [13]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [14]:
# To give you a preview -- calling Gemini with system and user messages:

import google.generativeai as genai
model = genai.GenerativeModel('models/gemini-2.5-flash-lite')
chat = model.start_chat(history=[])
response = chat.send_message(messages[-1]["content"])
print(response.text)

2 + 2 = 4


## And now let's build useful messages for Gemini, using a function

In [15]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [16]:
# Try this out, and then try for a few more websites

print(messages_for(ed))

[{'role': 'system', 'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'}, {'role': 'user', 'content': "You are looking at a website titled New Technique Auto-Selects Training Examples to Speed Up Fine-Tuning\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\n✨ New course! Enroll in\nBuilding and Evaluating Data Agents\nExplore Courses\nAI Newsletter\nThe Batch\nAndrew's Letter\nData Points\nML Research\nBlog\n✨ AI Dev x NYC\nCommunity\nForum\nEvents\nAmbassadors\nAmbassador Spotlight\nResources\nCompany\nAbout\nCareers\nContact\nStart Learning\nWeekly Issues\nAndrew's Letters\nData Points\nML Research\nBusiness\nScience\nCulture\nHardware\nAI Careers\nAbout\nSubscribe\nThe Batch\nMachine Learning Research\nArticle\nFaster Reinforcement Learning\nNe

## Time to bring it together - the API for OpenAI is very simple!

In [17]:
# And now: call the Gemini API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    chat = model.start_chat(history=[])
    response = chat.send_message(messages_for(website)[-1]["content"])
    return response.text

In [18]:
summarize("https://www.deeplearning.ai/the-batch/new-technique-auto-selects-training-examples-to-speed-up-fine-tuning/")

'## Summary: New Technique Auto-Selects Training Examples to Speed Up Fine-Tuning\n\nThis article introduces **GAIN-RL**, a new method developed by researchers at UC Berkeley and Duke University that significantly speeds up the fine-tuning of large language models (LLMs) using reinforcement learning.\n\nThe core innovation of GAIN-RL lies in its **automatic selection of training examples**. Instead of relying on external, often costly, heuristics to determine example importance, GAIN-RL utilizes the LLM\'s own internal signals – specifically, the **angles between vector representations of tokens**. The "angle concentration" of an example, derived from these token angles, directly correlates with the magnitude of gradient updates during training. Examples with higher angle concentration lead to larger updates and thus, faster learning.\n\n**How it works:**\n\n1.  **Calculate Angle Concentration:** A single forward pass over the entire dataset is performed to calculate the angle concentr

In [19]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [20]:
display_summary("https://www.deeplearning.ai/the-batch/new-technique-auto-selects-training-examples-to-speed-up-fine-tuning/")

Here's a summary of the website's content:

The website discusses a new technique called **GAIN-RL** developed by researchers at UC Berkeley and Duke University that significantly speeds up the process of fine-tuning large language models (LLMs) using reinforcement learning.

**Key Points:**

*   **Problem:** Fine-tuning LLMs with reinforcement learning is computationally expensive and time-consuming.
*   **Solution:** GAIN-RL automatically selects training examples based on the model's internal signals, specifically the angles between vector representations of tokens. Examples that result in larger gradient updates (higher "angle concentration") are prioritized.
*   **How it Works:** The method involves a single forward pass to calculate the angle concentration of each training example, sorting them from highest to lowest. The fine-tuning process then focuses on high-concentration examples first, gradually shifting to lower-concentration ones as the model learns.
*   **Results:** GAIN-RL generally accelerated learning by a factor of 2.5, achieving comparable performance in 70-80 training epochs to traditional methods requiring 200 epochs. For example, a Qwen 2.5 model reached 92.0% accuracy on GSM8K in 70 epochs with GAIN-RL, whereas the standard method needed 200 epochs.
*   **Significance:** This technique is more efficient than existing methods that often rely on external, costly heuristics (like human annotators or proprietary LLMs) to determine example difficulty. GAIN-RL uses a readily available internal model signal.

**News/Announcements:**

*   The website announces a new course: **"Building and Evaluating Data Agents"**.
*   It also highlights the availability of the **GAIN-RL code on GitHub**.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [22]:
display_summary("https://cnn.com")

This website is the homepage of CNN (Cable News Network), a major global news organization. It provides breaking news, the latest updates, and videos across a wide range of categories, including:

*   **General News:** US, World, Politics, Business, Health, Entertainment, Style, Travel, Sports, Science, Climate, and Weather.
*   **Specific Conflicts:** Ukraine-Russia War, Israel-Hamas War.
*   **Features and Series:** As Equals, Call to Earth, Freedom Project, Impact Your World, Inside Africa, CNN Heroes.
*   **Media:** Live TV, CNN Fast, Shows A-Z, CNN10, CNN Max, CNN TV Schedules, Podcasts.
*   **Interactive Content:** Games (crosswords, sudoku, quizzes).
*   **Account Services:** Options to sign in, manage accounts, settings, and subscribe to newsletters.

**Key News and Announcements Highlighted:**

*   **US Government Shutdown Fears:** Congress has a limited time to avert a shutdown, with "odds not looking good."
*   **Trump's Political Activities:** Mentions of Trump addressing the military, posting an AI-generated video, and his potential policies on "insane asylums."
*   **Afghanistan Internet Blackout:** A "total internet blackout" in Afghanistan has caused panic following Taliban vows to combat "immoral activities."
*   **Middle East Peace Efforts:** Analysis on Trump's new Middle East peace plan and the complexities of a Gaza ceasefire.
*   **Drone Presence in Korea:** The US is establishing a new Reaper drone unit near China.
*   **"Dumbphone" Comeback:** The article notes a resurgence of simpler mobile phones, described as "more exclusive and expensive."
*   **Celebrity News:** Reports on the separation of Nicole Kidman and Keith Urban, Michelle Pfeiffer becoming a grandmother, and Taylor Swift's significant numbers.
*   **Environmental Issues:** Namibia is deploying the army to combat a devastating wildfire in a game reserve.
*   **Crime and Investigations:** An ongoing investigation into a shooting at a Michigan church and legal proceedings against Sean "Diddy" Combs.
*   **Economic Indicators:** Non-scientific indicators like hot dogs and camping trips are being used to gauge the economy.
*   **Health News:** A report on the "devastating impact" of US abortion restrictions and new findings on heart disease detection.
*   **Sports News:** Discussion of a major soccer team's long-distance travel for a Champions League match and a boxer's encounter with police.
*   **Arts and Culture:** Features on "Made in Italy" fashion and unique travel destinations.
*   **Global Events:** Mysterious UAV sightings in Denmark and the ongoing impact of historical events like "I am Charlie."

In [ ]:
display_summary("https://anthropic.com")

In [29]:
#create a simple UI

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import threading

# %% cell 31 code

# Initialize the Gemini model for the UI
model = genai.GenerativeModel('models/gemini-2.5-flash-lite')

# Enhanced CSS styling for modern UI - displayed separately
from IPython.display import HTML
css_styles = HTML("""
<style>
/* Modern card-based design */
.summary-card {
    background: #ffffff;
    border-radius: 12px;
    box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1), 0 2px 4px -1px rgba(0, 0, 0, 0.06);
    padding: 24px;
    margin: 16px 0;
    border: 1px solid #e5e7eb;
}

/* Enhanced header styling */
.summary-header {
    color: #1f2937;
    font-size: 28px;
    font-weight: 700;
    margin-bottom: 8px;
    text-align: center;
}

.summary-subtitle {
    color: #6b7280;
    font-size: 14px;
    margin-bottom: 24px;
    text-align: center;
}

/* Form section styling */
.form-section {
    background: #f9fafb;
    border-radius: 8px;
    padding: 20px;
    margin-bottom: 20px;
    border-left: 4px solid #3b82f6;
}

.form-label {
    color: #374151;
    font-weight: 600;
    margin-bottom: 8px;
    display: block;
}

/* Enhanced input styling */
.url-input-container {
    margin-bottom: 16px;
}

.url-input {
    width: 100%;
    padding: 12px 16px;
    border: 2px solid #d1d5db;
    border-radius: 8px;
    font-size: 16px;
    transition: border-color 0.2s ease;
}

.url-input:focus {
    outline: none;
    border-color: #3b82f6;
    box-shadow: 0 0 0 3px rgba(59, 130, 246, 0.1);
}

/* Button styling */
.button-container {
    display: flex;
    gap: 12px;
    justify-content: center;
    margin: 20px 0;
}

.primary-button {
    background: linear-gradient(135deg, #3b82f6, #1d4ed8);
    color: white;
    border: none;
    padding: 12px 32px;
    border-radius: 8px;
    font-weight: 600;
    cursor: pointer;
    transition: all 0.2s ease;
    box-shadow: 0 2px 4px rgba(59, 130, 246, 0.2);
}

.primary-button:hover {
    background: linear-gradient(135deg, #2563eb, #1e40af);
    transform: translateY(-1px);
    box-shadow: 0 4px 8px rgba(59, 130, 246, 0.3);
}

.primary-button:disabled {
    background: #9ca3af;
    cursor: not-allowed;
    transform: none;
    box-shadow: none;
}

/* Status and progress styling */
.status-section {
    background: #fefce8;
    border: 1px solid #fbbf24;
    border-radius: 8px;
    padding: 12px 16px;
    margin: 16px 0;
}

.status-text {
    color: #92400e;
    font-size: 14px;
    margin: 0;
}

.progress-section {
    background: #f0f9ff;
    border: 1px solid #0ea5e9;
    border-radius: 8px;
    padding: 12px 16px;
    margin: 16px 0;
}

.progress-text {
    color: #0c4a6e;
    font-size: 14px;
    margin: 0;
}

/* Output section styling */
.output-section {
    background: #f8fafc;
    border-radius: 8px;
    padding: 20px;
    margin-top: 20px;
    border: 1px solid #e2e8f0;
}

.output-header {
    color: #334155;
    font-size: 18px;
    font-weight: 600;
    margin-bottom: 16px;
    border-bottom: 2px solid #e2e8f0;
    padding-bottom: 8px;
}

/* Style the summary output area */
.summary-output-container {
    background: #f8fafc;
    border-radius: 8px;
    padding: 20px;
    margin-top: 20px;
    border: 1px solid #e2e8f0;
    min-height: 100px;
}

/* Error styling */
.error-section {
    background: #fef2f2;
    border: 1px solid #f87171;
    border-radius: 8px;
    padding: 12px 16px;
    margin: 16px 0;
}

.error-text {
    color: #dc2626;
    font-size: 14px;
    margin: 0;
}

/* Success styling */
.success-section {
    background: #f0fdf4;
    border: 1px solid #22c55e;
    border-radius: 8px;
    padding: 12px 16px;
    margin: 16px 0;
}

.success-text {
    color: #15803d;
    font-size: 14px;
    margin: 0;
}

/* Responsive design */
@media (max-width: 768px) {
    .summary-card {
        padding: 16px;
        margin: 8px 0;
    }

    .button-container {
        flex-direction: column;
    }

    .primary-button {
        width: 100%;
    }
}
</style>
""")

# Display CSS styles first
display(css_styles)

# Create UI elements

# Header section with logo
logo_html = widgets.HTML('''
<div style="text-align: center; margin-bottom: 16px;">
    <img src="https://img.icons8.com/fluency/96/000000/web.png" alt="Website Summarizer Logo"
         style="width: 64px; height: 64px; margin-bottom: 8px; border-radius: 50%;">
</div>
''')

header = widgets.HTML('<div class="summary-card"><h1 class="summary-header">🌐 Website Summarizer</h1><p class="summary-subtitle">Powered by Google\'s Gemini AI</p>')

# Form section
form_section = widgets.HTML('<div class="form-section"><label class="form-label">Website URL</label><div class="url-input-container">')

url_input = widgets.Text(
    value='',
    placeholder='https://example.com',
    description='',
    disabled=False,
    layout=widgets.Layout(width='100%')
)

# Button section
button_section = widgets.HTML('<div class="button-container">')
submit_button = widgets.Button(
    description='🚀 Summarize Website',
    disabled=False,
    button_style='primary',
    tooltip='Click to summarize the website',
    layout=widgets.Layout(width='auto')
)
button_section_close = widgets.HTML('</div></div>')

# Status and progress sections
status_section = widgets.HTML('<div class="status-section" style="display: none;"><p class="status-text">Enter a URL and click Summarize</p></div>')
progress_section = widgets.HTML('<div class="progress-section" style="display: none;"><p class="progress-text">⏳ Processing...</p></div>')

# Output section container
output_container = widgets.HTML('<div class="summary-output-container" style="display: none;">')
summary_output = widgets.Output()
output_close = widgets.HTML('</div>')

def on_button_click(b):
    url = url_input.value.strip()
    if not url:
        # Show error state
        status_section.value = '<div class="status-section"><p class="error-text">❌ Please enter a valid URL.</p></div>'
        progress_section.value = '<div class="progress-section" style="display: none;"><p class="progress-text">⏳ Processing...</p></div>'
        output_container.value = '<div class="summary-output-container" style="display: none;">'
        return

    # Show processing state
    status_section.value = '<div class="status-section" style="display: none;"><p class="status-text">Enter a URL and click Summarize</p></div>'
    progress_section.value = '<div class="progress-section"><p class="progress-text">⏳ Analyzing website content...</p></div>'
    output_container.value = '<div class="summary-output-container" style="display: none;">'
    submit_button.disabled = True

    # Run summarization in a separate thread to avoid blocking
    def summarize_in_thread():
        try:
            summary = summarize(url)

            # Show success state
            progress_section.value = '<div class="progress-section"><p class="progress-text">✅ Summary generated successfully!</p></div>'
            status_section.value = f'<div class="status-section"><p class="success-text">📝 Summary ready for: {url}</p></div>'

            # Show output section with summary and website info
            output_container.value = '<div class="summary-output-container">'

            # Display website favicon and basic info
            website = Website(url)
            favicon_html = ""
            if website.favicon:
                favicon_html = f'''
                <div style="text-align: center; margin-bottom: 16px; padding: 12px; background: #f8fafc; border-radius: 8px; border: 1px solid #e2e8f0;">
                    <img src="{website.favicon}" alt="Website favicon" style="width: 32px; height: 32px; margin-right: 8px; vertical-align: middle; border-radius: 4px;">
                    <strong style="color: #374151;">{website.title}</strong>
                </div>
                '''

            # Display main images if available
            images_html = ""
            if website.images:
                images_html = '<div style="margin: 16px 0; text-align: center;">'
                for i, img in enumerate(website.images[:2]):  # Show max 2 images
                    images_html += f'''
                    <div style="display: inline-block; margin: 8px; text-align: center;">
                        <img src="{img['url']}" alt="{img['alt']}" style="max-width: 200px; max-height: 150px; border-radius: 8px; border: 1px solid #e2e8f0;">
                        <p style="font-size: 12px; color: #6b7280; margin-top: 4px; max-width: 200px; overflow: hidden; text-overflow: ellipsis; white-space: nowrap;">{img['alt'] or 'Website image'}</p>
                    </div>
                    '''
                images_html += '</div>'

            # Display the summary in the output widget
            with summary_output:
                clear_output()
                # Show favicon and images first
                if favicon_html or images_html:
                    display(HTML(favicon_html + images_html))
                # Then show the summary
                display(Markdown(summary))

        except Exception as e:
            # Show error state
            progress_section.value = '<div class="progress-section"><p class="progress-text">❌ Error occurred during summarization</p></div>'
            status_section.value = f'<div class="status-section"><p class="error-text">⚠️ Could not summarize: {url}. Please check the URL and try again.</p></div>'

            # Show output section with error details
            output_container.value = '<div class="summary-output-container">'

            with summary_output:
                clear_output()
                error_details = widgets.HTML(f'<p style="color: #6b7280; font-size: 12px; margin-top: 8px;">Error details: {str(e)}</p>')
                display(error_details)

        finally:
            submit_button.disabled = False

    thread = threading.Thread(target=summarize_in_thread)
    thread.daemon = True
    thread.start()

submit_button.on_click(on_button_click)

# Create layout - modern card-based design
ui = widgets.VBox([
    # Logo and main content sections
    logo_html,
    header,
    form_section,
    url_input,
    button_section,
    submit_button,
    button_section_close,

    # Status and progress sections
    status_section,
    progress_section,

    # Output section
    output_container,
    summary_output,
    output_close
])

# Display the UI
display(ui)


# %% cell 32 markdown

# ## How to use:
# 
# 1. Enter a website URL in the text field
# 2. Click the "Summarize" button
# 3. Wait for the summary to appear below
# 
# The UI will show progress and status messages. This works with the same websites as the original function (no JavaScript-heavy sites or CloudFront-protected sites).
